In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain transformers accelerate torch

In [0]:
dbutils.library.restartPython()

### Build an AI Agent which can write code for a specified
use case based on specified goals

In [0]:
import os
import sys
import asyncio
import nest_asyncio
from typing import Optional, List

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor


from databricks_langchain import ChatDatabricks 
from langgraph.graph import StateGraph, END  # pip install langgraph


In [0]:
# -----------------------
# 1) Environment / LLM
# -----------------------
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-1-8b-instruct" # Model Serving endpoint name; other option see "Serving" under AI/ML tab (e.g. databricks-gpt-oss-20b)

llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.2)

In [0]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Good lightweight defaults; change if you want a different model
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"   # try 0.5B/1.5B/3B depending on your cluster
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
DEVICE = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    device_map="auto" if DEVICE == 0 else None,
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=DEVICE,
)

In [0]:
def hf_chat(prompt: str, temperature: float = 0.3, max_new_tokens: int = 800) -> str:
    try:
        messages = [{"role": "user", "content": prompt}]
        chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    except Exception:
        chat_prompt = prompt

    do_sample = temperature > 0

    kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        return_full_text=False,
    )

    # Only pass sampling params if sampling is enabled
    if do_sample:
        kwargs.update(dict(
            temperature=temperature,
            top_p=0.9,
        ))

    out = gen(chat_prompt, **kwargs)[0]["generated_text"]
    return out.strip()

In [0]:
# ---------- Utility Functions ----------
def generate_prompt(use_case: str, goals: list[str], previous_code: str = "", feedback: str = "") -> str:
    base_prompt = f"""
You are an AI coding agent. Your job is to write Python code based on the following use case:

Use Case: {use_case}

Your goals are:
{chr(10).join(f"- {g.strip()}" for g in goals)}
"""
    if previous_code:
        base_prompt += f"\nPreviously generated code:\n{previous_code}\n"
    if feedback:
        base_prompt += f"\nFeedback on previous version:\n{feedback}\n"

    base_prompt += "\nReturn ONLY valid Python code. No markdown fences. No explanations."
    return base_prompt

In [0]:
def get_code_feedback(code: str, goals: list[str]) -> str:
    feedback_prompt = f"""
You are a Python code reviewer. A code snippet is shown below. Based on the following goals:

{chr(10).join(f"- {g.strip()}" for g in goals)}

Critique this code and identify if the goals are met. Mention improvements for clarity, simplicity,
correctness, edge case handling, and test coverage (if requested by goals).

Code:
{code}
"""
    return hf_chat(feedback_prompt, temperature=0.2, max_new_tokens=500)


In [0]:
def goals_met(feedback_text: str, goals: list[str]) -> bool:
    review_prompt = f"""
You are an AI reviewer.

Goals:
{chr(10).join(f"- {g.strip()}" for g in goals)}

Feedback:
\"\"\"
{feedback_text}
\"\"\"

Based on the feedback above, have the goals been met?

Respond with only one word: True or False.
"""
    resp = hf_chat(review_prompt, temperature=0.0, max_new_tokens=5).strip().lower()
    # be robust to extra tokens
    return resp.startswith("true")

In [0]:
def clean_code_block(code: str) -> str:
    # remove accidental markdown fences if the model outputs them
    lines = code.strip().splitlines()
    if lines and lines[0].strip().startswith("```"):
        lines = lines[1:]
    if lines and lines[-1].strip().startswith("```"):
        lines = lines[:-1]
    return "\n".join(lines).strip()

In [0]:
def add_comment_header(code: str, use_case: str) -> str:
    comment = f"# This Python program implements the following use case:\n# {use_case.strip()}\n"
    return comment + "\n" + code


In [0]:
from pathlib import Path
import random
import re

def save_code_local(code: str, use_case: str) -> str:
    base_dir = Path("/tmp/mcp_code_agent")
    base_dir.mkdir(parents=True, exist_ok=True)

    short = re.sub(r"[^a-zA-Z0-9_]", "", use_case.lower().replace(" ", "_"))[:10]
    short = short or "code"
    filename = f"{short}_{random.randint(1000,9999)}.py"

    path = base_dir / filename
    path.write_text(code, encoding="utf-8")

    return str(path)

In [0]:
# ---------- Main Agent ----------
def run_code_agent(use_case: str, goals_input: str, max_iterations: int = 5) -> str:
    goals = [g.strip() for g in goals_input.split(",") if g.strip()]

    previous_code = ""
    feedback_text = ""
    code = ""

    for i in range(max_iterations):
        prompt = generate_prompt(use_case, goals, previous_code, feedback_text)
        raw_code = hf_chat(prompt, temperature=0.3, max_new_tokens=900)
        code = clean_code_block(raw_code)

        feedback_text = get_code_feedback(code, goals)

        if goals_met(feedback_text, goals):
            break

        previous_code = code

    final_code = add_comment_header(code, use_case)
    out_path = save_code_local(final_code, use_case)
    print("Saved to:", out_path)

In [0]:
use_case_input = "Write code to find BinaryGap of a given positive integer"
goals_input = "Code simple to understand, Functionally correct, Handles comprehensive edge cases, Takes positive integer input only, prints the results with few examples"

out_path = run_code_agent(use_case_input, goals_input, max_iterations=5)
print("Saved to:", out_path)


In [0]:
import os
os.listdir("/tmp/mcp_code_agent")

In [0]:
path = "/tmp/mcp_code_agent/write_code_7058.py"

with open(path, "r", encoding="utf-8") as f:
    print(f.read())